In [1]:
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from imutils import paths

import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
import os

2022-04-20 11:56:13.269599: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-20 11:56:13.269796: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
class modelNet:
    @staticmethod
    def build(width, height, depth, classes):
        model = Sequential()
        inputShape = (height, width, depth)
        
        model.add(Conv2D(20, (5, 5), padding="same", input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
        
        model.add(Conv2D(50, (5, 5), padding="same"))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        
        model.add(Flatten())
        model.add(Dense(10))
        model.add(Activation("relu"))
        
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        
        return model

In [3]:
EPOCHS = 15
INIT_LR = 1e-3
BS = 64

In [4]:
print("[INFO] loading images...")
data = []
labels = []

[INFO] loading images...


In [5]:
imagePaths = sorted(list(paths.list_images("Dataset")))
random.seed(42)
random.shuffle(imagePaths)

In [6]:
for imagePath in imagePaths:    
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)
    
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (28, 28))
    image = img_to_array(image)
    data.append(image)
    

In [7]:
data[0].shape

(28, 28, 3)

In [8]:
unique, counts = np.unique(np.array(labels), return_counts=True)
dict(zip(unique, counts))

{0: 1973}

In [9]:
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [10]:
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)

trainY = to_categorical(trainY, num_classes=2)
testY = to_categorical(testY, num_classes=2)

In [11]:
print("[INFO] compiling model...")
model = modelNet.build(width=28, height=28, depth=3, classes=2)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)

model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

[INFO] compiling model...


2022-04-20 11:56:35.224982: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/awiros-tech/anaconda3/envs/tripleRider/lib/python3.8/site-packages/cv2/../../lib64:
2022-04-20 11:56:35.225176: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-20 11:56:35.225238: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (awi-stavya): /proc/driver/nvidia/version does not exist
2022-04-20 11:56:35.229062: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/hom

In [12]:
print("[INFO] training network...")
H = model.fit(trainX, trainY, batch_size=BS,
	validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS, verbose=1)
# save the model to disk
print("[INFO] serializing network...")
model.save("model_1.h5", save_format="h5")

[INFO] training network...
Epoch 1/15


In [ ]:
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Double and triple rider")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot_3.png")

In [ ]:
predictions = model.predict(x=testX, batch_size=32)

print(predictions.shape)

(2417, 2)


In [ ]:
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1),target_names=["triple_rider","double_rider"]))

              precision    recall  f1-score   support

triple_rider       0.93      0.86      0.90      1269
double_rider       0.86      0.93      0.90      1148

    accuracy                           0.90      2417
   macro avg       0.90      0.90      0.90      2417
weighted avg       0.90      0.90      0.90      2417



In [ ]:
#importing the original dataset
data = []
labels = []
imagePaths = sorted(list(paths.list_images("Dataset")))

for imagePath in imagePaths:
    
    label = imagePath.split(os.path.sep)[-2]
    if label == "triple_person":
        label = 1
    else:
        label = 0
    labels.append(label)
    
    # for images
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (28, 28))
    image = img_to_array(image)
    data.append(image)


In [ ]:
data = np.array(data)
labels = np.array(labels)

labels = to_categorical(labels, num_classes=2)

In [ ]:
predictions = model.predict(x=data, batch_size=64)


In [ ]:
print(classification_report(labels.argmax(axis=1), predictions.argmax(axis=1),
                        target_names=["double_rider","triple_rider"]))

              precision    recall  f1-score   support

double_rider       0.91      0.52      0.67       948
triple_rider       0.03      0.22      0.05        63

    accuracy                           0.51      1011
   macro avg       0.47      0.37      0.36      1011
weighted avg       0.86      0.51      0.63      1011

